In [1]:
from cosecha_colectiva import miscelaneous as m
import pandas as pd
m.try_find_conf_file()


from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import config
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import test_cosecha_api as tca
from cosecha_colectiva import CAF_DB_Monitor as CDM

from cosecha_colectiva import miscelaneous as m

import logging
import pathlib
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


[2025-03-12 22:16:41,370][INFO]: Connecting root@junction.proxy.rlwy.net:19994
[2025-03-12 22:16:42,591][INFO]: Connected root@junction.proxy.rlwy.net:19994


In [2]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'CreacionGrupo.log')
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.INFO)

In [3]:
xls_name = 'Verde_Futuro_Test.xlsx'
xls_name = 'Verde_Futuro.xlsx'

xls_name = 'Tropa Gusgones_2.0.xlsx'
xls_name = 'Tropa Gusgones_2.0_Test.xlsx'

xls_name = 'Alcancía Viva.xlsx'
#xls_name = 'Alcancía Viva Test.xlsx'

xls_name = 'Licenciados_Asociados.xlsx'
#xls_name = 'Licenciados_Asociados_Test.xlsx'

xls_name = 'Juntas_ahorramos.xlsx'
xls_name = 'Juntes_ahorramos.xlsx'
#xls_name = 'Juntos_ahorramos.xlsx'



#xls_name = 'CAF_GrupoTest.xlsx'
#xls_name = 'CAF_GrupoHidalgo.xlsx'
#xls_name = 'GrupoCafam.xlsx'

#xls_name = 'Yeiyei'

In [4]:
xls_name

'Juntes_ahorramos.xlsx'

In [5]:
[id_grupo, grupo_found] = qc.get_grupo_id_by_name(xls_name.replace(".xlsx", ""))
print(id_grupo)
#qc.delete_grupo(id_grupo, solo_sesiones=True)
#qc.delete_grupo(id_grupo)

2616


In [6]:
if not grupo_found:
    grupo_data = m.read_caf_excel_sheet(pathlib.Path(xls_name), 'Grupo')
    id_grupo = qc.insert_grupo(grupo_data.to_dict('records'))
    id_grupo = id_grupo['Grupo_id']
    
id_grupo

2616

In [7]:
if not grupo_found:

    socios_xls = pd.DataFrame(m.get_dict_usuarios_xls(xls_name))
    # Solo crea socios cuando viene los datos completos
    if not socios_xls['Fecha_nac'].isna().all():
        print('Creando usuarios')
        tca.CAF_API_group_creation_tester.create_users(xls_name)

    grupo_socio_df = qc.prepare_insert_socio_grupo(xls_name, id_grupo)

    df_acuerdos = m.read_caf_excel_sheet(pathlib.Path(xls_name), "Acuerdos")
    df_acuerdos['Grupo_id'] = id_grupo
    df_acuerdos['Fecha_acuerdos'] = datetime.strftime(date.today(), format='%Y-%m-%d')
    df_acuerdos['Status'] = 1

    df_acuerdos['Id_socio_administrador'] = grupo_socio_df.loc[0,'Socio_id']
    df_acuerdos['Id_socio_administrador_suplente'] = grupo_socio_df.loc[1,'Socio_id']
    
    qc.insert_acuerdos(df_acuerdos.to_dict('records')[0])

    
    


In [8]:
qc.delete_grupo(id_grupo, solo_sesiones=True, force_delete=True, desde_sesion_0=True)


#qc.restart_grupo_acciones(id_grupo)

([], 'transaccion_prestamos')
([], 'transacciones')
([], 'interes_prestamo')
([], 'ganancias')
([], 'multas')
([], 'prestamos')
([], 'prestamos')
([{'Asistencia_id': 137899}, {'Asistencia_id': 137900}, {'Asistencia_id': 137901}, {'Asistencia_id': 137902}, {'Asistencia_id': 137903}, {'Asistencia_id': 137904}, {'Asistencia_id': 137905}, {'Asistencia_id': 137906}, {'Asistencia_id': 137907}, {'Asistencia_id': 137908}, {'Asistencia_id': 137909}, {'Asistencia_id': 137910}, {'Asistencia_id': 137911}, {'Asistencia_id': 137912}, {'Asistencia_id': 137913}, {'Asistencia_id': 137914}, {'Asistencia_id': 137915}, {'Asistencia_id': 137916}, {'Asistencia_id': 137917}, {'Asistencia_id': 137918}, {'Asistencia_id': 137919}, {'Asistencia_id': 137920}, {'Asistencia_id': 137921}, {'Asistencia_id': 137922}, {'Asistencia_id': 137923}, {'Asistencia_id': 137924}, {'Asistencia_id': 137925}, {'Asistencia_id': 137926}, {'Asistencia_id': 137927}, {'Asistencia_id': 137928}, {'Asistencia_id': 137929}, {'Asistencia_id

In [9]:
start_month = 1
end_month = 5
initial_date = (date.today() - relativedelta(months=end_month))

monitor_grupo = CDM.CAF_DB_Monitor(id_grupo, xls_name, starting_date=initial_date)
monitor_grupo.socios_acciones


,Grupo_socio_id,Socio_id,Tipo_socio,Status,Acciones,Grupo_id,CURP,xls_order
0,9081,3018,ADMIN,1,0,2616,JUEA700101MDFNNN01,0
1,9082,3019,SUPLENTE,1,0,2616,JUEA700101MDFNNN02,1
2,9083,3020,SOCIO,1,0,2616,JUEA700101MDFNNN03,2
3,9084,3021,SOCIO,1,0,2616,JUEA700101MDFNNN04,3
4,9085,3022,SOCIO,1,0,2616,JUEA700101MDFNNN05,4
5,9086,3023,SOCIO,1,0,2616,JUEA700101MDFNNN06,5
6,9087,3024,SOCIO,1,0,2616,JUEA700101MDFNNN07,6
7,9088,3025,SOCIO,1,0,2616,JUEA700101MDFNNN08,7
8,9089,3026,SOCIO,1,0,2616,JUEA700101MDFNNN09,8


In [10]:

for month in range(start_month, end_month+1):

    print('month: ', month)

    try:
        monitor_grupo.actualiza_todo_sesion(xls_name, month, type_xls='NEW', bd_update=True)
    except Exception as e:
        print(e)
        raise(e)
        break

month:  1
month:  2
month:  3
month:  4
month:  5


In [11]:
monitor_grupo.socios_acciones

,Grupo_socio_id,Socio_id,Grupo_id,Tipo_socio,Acciones,Status,CURP
0,9081,3018,2616,ADMIN,880,1,JUEA700101MDFNNN01
1,9082,3019,2616,SUPLENTE,360,1,JUEA700101MDFNNN02
2,9083,3020,2616,SOCIO,280,1,JUEA700101MDFNNN03
3,9084,3021,2616,SOCIO,480,1,JUEA700101MDFNNN04
4,9085,3022,2616,SOCIO,520,1,JUEA700101MDFNNN05
5,9086,3023,2616,SOCIO,1120,1,JUEA700101MDFNNN06
6,9087,3024,2616,SOCIO,760,1,JUEA700101MDFNNN07
7,9088,3025,2616,SOCIO,520,1,JUEA700101MDFNNN08
8,9089,3026,2616,SOCIO,280,1,JUEA700101MDFNNN09


In [12]:
monitor_grupo.prestamos

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Fecha_inicial,Fecha_final,Estatus_ampliacion,Observaciones,Num_sesiones,...,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono,Status_socio
0,6555,1000.0,360.0,40.0,40.0,2024-10-12,2025-05-12,False,Préstamo generado en script automático,7,...,1,1.5,1,1.5,10.0,10.0,10.0,90.0,100.0,1
1,6556,500.0,350.0,20.0,20.0,2024-10-12,2025-05-12,False,Préstamo generado en script automático,7,...,1,1.5,1,1.5,5.0,5.0,5.0,95.0,100.0,1


In [13]:
monitor_grupo.sesiones_bd

,Sesion_id,Fecha,Activa,Caja,Acciones,Ganancias,Fecha_prox_reunion,Lugar_prox_reunion,Tipo_sesion,Grupo_id,created_at
0,19784,2024-10-12,0,2630.0,4100,28.0,2024-11-12,Sesion creada en script,1,2616,2024-10-12
1,19785,2024-11-12,0,3050.0,4240,141.0,2024-12-12,Sesion creada en script,1,2616,2024-11-12
2,19786,2024-12-12,0,3700.0,4650,204.0,2025-01-12,Sesion creada en script,1,2616,2024-12-12
3,19787,2025-01-12,0,4040.0,4790,217.5,2025-02-12,Sesion creada en script,1,2616,2025-01-12
4,19788,2025-02-12,0,4650.0,5200,231.0,2025-03-12,Sesion creada en script,1,2616,2025-02-12


In [ ]:
monitor_grupo.caja

4650.0

In [ ]:
monitor_grupo.ganancias

,Ganancias_id,Ganancia_accion,Monto_ganancia,Entregada,Socio_id,Sesion_id,periodo
0,79912,0.007317,1.0,0,3020,19777,nuevo
1,79913,0.007317,1.0,0,3026,19777,nuevo
2,79914,0.007317,2.0,0,3019,19777,nuevo
3,79915,0.007317,2.5,0,3021,19777,nuevo
4,79916,0.007317,2.5,0,3022,19777,nuevo
5,79917,0.007317,2.5,0,3025,19777,nuevo
6,79918,0.007317,4.0,0,3024,19777,nuevo
7,79919,0.007317,5.5,0,3018,19777,nuevo
8,79920,0.007317,7.0,0,3023,19777,nuevo
9,79921,0.027123,5.5,0,3020,19778,nuevo


In [ ]:
monitor_grupo.ganancias_acum

,Monto_ganancia
Socio_id,
3018,41.5
3019,16.5
3020,10.5
3021,21.0
3022,22.5
3023,52.0
3024,34.0
3025,22.5
3026,10.5


In [ ]:
monitor_grupo.socios_acciones

,Grupo_socio_id,Socio_id,Grupo_id,Tipo_socio,Acciones,Status,CURP
0,9081,3018,2616,ADMIN,880,1,JUEA700101MDFNNN01
1,9082,3019,2616,SUPLENTE,360,1,JUEA700101MDFNNN02
2,9083,3020,2616,SOCIO,280,1,JUEA700101MDFNNN03
3,9084,3021,2616,SOCIO,480,1,JUEA700101MDFNNN04
4,9085,3022,2616,SOCIO,520,1,JUEA700101MDFNNN05
5,9086,3023,2616,SOCIO,1120,1,JUEA700101MDFNNN06
6,9087,3024,2616,SOCIO,760,1,JUEA700101MDFNNN07
7,9088,3025,2616,SOCIO,520,1,JUEA700101MDFNNN08
8,9089,3026,2616,SOCIO,280,1,JUEA700101MDFNNN09
